# Challenge 06/12/2024

Challenge instructions [here](https://adventofcode.com/2024/day/6)

## Highlights & Notes

- Goal: Determine a number of places where the guard is gonna go


## Setup & Imports


In [1]:
import numpy as np
import sys
import os
import time

# Get utility functions
sys.path.append(os.path.abspath('../utils'))
from utils import read_input_file, remove_newline_char, split_lines

# Quick ANSI color code shortcuts
r = "\033[31m";y = "\033[33m";g = "\033[32m";b = "\033[34m";e = "\033[0m"

## Part 1


In [7]:
def parse_input_as_2d_grid(file_name):
    return np.array([list(x) for x in remove_newline_char(read_input_file(file_name))])

def find_guard_pos_and_direction(map):
    for i in range(len(map)):
        for j in range(len(map[i])):
            if map[i][j] in directions:
                return (i, j, map[i][j])
    raise Exception("No guard found in the map")

def is_obstacle(pos, grid):
    row, col = pos
    return grid[row][col] == '#'

def turn_right(current_direction):
    directions_order = ['^', '>', 'v', '<']
    idx = directions_order.index(current_direction)
    return directions_order[(idx + 1) % 4]

def next_guard_move(current_pos, grid):
    row, col, direction = current_pos
    dr, dc = directions[direction]
    next_row, next_col = row + dr, col + dc

    # Check if next position is within grid boundaries
    if 0 <= next_row < len(grid) and 0 <= next_col < len(grid[0]):
        if is_obstacle((next_row, next_col), grid):
            # Turn right and stay in the same position
            new_direction = turn_right(direction)
            return (row, col, new_direction)
        else:
            # Move forward
            return (next_row, next_col, direction)
    else:
        # Move off the grid
        return (next_row, next_col, direction)

directions = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1)
}

grid = parse_input_as_2d_grid("input.txt")
startGuardPos = find_guard_pos_and_direction(grid)
print(f"Initial guard position: {startGuardPos}")

visitedLocations = set()
guardPos = startGuardPos
visitedLocations.add((guardPos[0], guardPos[1]))
while True:
    guardPos = next_guard_move(guardPos, grid)
    row, col, direction = guardPos

    # **Check if guard has moved off the grid**
    if not (0 <= row < len(grid) and 0 <= col < len(grid[0])):
        break  # Guard has left the mapped area

    visitedLocations.add((row, col))
    # print(f"Guard position: {guardPos}")
    # time.sleep(0.5)

print(f"Number of distinct positions visited: {len(visitedLocations)}")

Initial guard position: (65, 85, '^')
Number of distinct positions visited: 5030


## Part 2

Instructions [here](https://adventofcode.com/2024/day/6#part2)


In [8]:
# Part 2: Only consider obstruction positions from visited_locations
possibleObsPos = [
    (row, col)
    for row, col in visitedLocations
    if grid[row][col] == '.' and (row, col) != (startGuardPos[0], startGuardPos[1])
]

nbNewPossibleObs = 0

for obsPos in possibleObsPos:
    # Create a deep copy of the grid and add the obstruction
    gridCopy = [row.copy() for row in grid]
    gridCopy[obsPos[0]][obsPos[1]] = '#'
    
    # Initialize the guard's state
    guardState = startGuardPos
    visitedStates = set()
    visitedStates.add(guardState)
    
    while True:
        guardState = next_guard_move(guardState, gridCopy)
        row, col, direction = guardState

        # If guard has moved off the grid, no loop occurs
        if not (0 <= row < len(gridCopy) and 0 <= col < len(gridCopy[0])):
            break

        if guardState in visitedStates:
            # Loop detected
            nbNewPossibleObs += 1
            break
        else:
            visitedStates.add(guardState)

print(f"Number of different positions where a new obstruction would cause a loop: {nbNewPossibleObs}")

Number of different positions where a new obstruction would cause a loop: 1928
